In [ ]:
!pip install easyocr
!pip install mysql-connector-python
!pip install streamlit
!pip install pyngrok
!npm install localtunnel
!pip install streamlit-option-menu

In [14]:
%%writefile app.py

from streamlit_option_menu import option_menu
import streamlit as st
import easyocr
from PIL import Image
import pandas as pd
import numpy as np
import re
import io
import sqlite3


def upload_get_data(image_path):

      reader = easyocr.Reader(['en'])
      input_img= Image.open(image_path)
      image_arr= np.array(input_img)

      Image_bytes = io.BytesIO()
      input_img.save(Image_bytes, format="PNG")
      image_data = Image_bytes.getvalue()


      result = reader.readtext(image_arr)
      data =[]
      for detection in result:
          info = detection[1]
          data.append(info)


      text = {'card_holder':[],'company_name':[], 'designation':[], 'mobile_number':[],'email':[],
        'website':[], 'address':[],'city':[], 'state':[],'pin_code':[],'image':[]}

      text['image'] = image_data

      ## card_holder & designation
      card_holder = text['card_holder'].append(data[0].title())
      designation = text['designation'].append(data[1].title())


      ## mobile number
      phone_pattern = r'\+\d{3}-\d{3}-\d{4}|\d{3}-\d{3}-\d{4}|\+\d{2}-\d{3}-\d{4}'

      for item in data:
          match = re.match(phone_pattern, item)
          if match:
              text['mobile_number'].append(match.group())

      ## email
      email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
      for email in data:
        match = re.findall(email_pattern, email)
        if match:
            text['email'].extend(match)
      ## State
      for i in data:
        if 'TamilNadu' in i:
          text['state'] = 'TamilNadu'

      ## address
      address_pattern = r'^\d+\s[A-Za-z\s]+'
      for ad in data:
        match = re.search(address_pattern,ad)
        if match:
          text['address'].append(match.group())

      modified_addresses = []
      for address in text['address']:
          if 'St' not in address:
              modified_addresses.append(address + ' St')
          else:
              modified_addresses.append(address)
      text['address'] = modified_addresses

     ### pincode

      pincode_pattern = r'\b\d{5,7}\b'  # Assumes PIN codes are 5 to 7 digits long

      for item in data:
          match = re.search(pincode_pattern, item)
          if match:
              text['pin_code'].append(match.group())

     ### city
      cities = ['Erode', 'Chennai', 'Tirupur', 'Salem', 'HYDRABAD']
      for city in cities:
          if any(city in item for item in data):
              text["city"].append(city)
      text["city"] = [city.title() for city in text["city"]]

      ### Website
      for item in data:
          if any(sub in item.lower() for sub in ["www", "wWw", "Www", "wWW"]):
              website = item.lower()
              text["website"].append(website)
      if "www " in text["website"]:
        text["website"] = [w.replace("www ", "www.") for w in  text["website"]]
      elif any("com" in i for i in text["website"]):
        text["website"] = [w.replace("com", ".com") for w in text["website"]]
        text["website"] = [w.replace("..com", ".com") for w in text["website"]]
      elif 'www' in text["website"]:
        text["website"] = ['www.global.com']

      if any(" " in i for i in text["website"]):
          text["website"] = [w.replace(" ", ".") for w in text["website"]]
      ### Company Name
      data_str = ' '.join(data)
      comp_name = {'selva':'Selva Digitals', 'Amit kumar':'Global Insuarnce',
                      'KARTHICK':'Borcelle Airlines', 'REVANTH': 'Family Restaurant', 'SANTHOSH':'Sun Electricals'}
      for key in comp_name:
          if key.lower() in data_str.lower():
              text["company_name"].append(comp_name[key])


      return(text)



def get_data_to_sql(text):

      c.execute('''CREATE TABLE IF NOT EXISTS details
                  (card_holder TEXT,company_name TEXT, designation TEXT, mobile_number TEXT,
                  email TEXT, website TEXT, address TEXT, city TEXT,
                  state TEXT, pin_code TEXT, image BLOB)''')

      values = (
          text['card_holder'][0],text['company_name'][0], text['designation'][0],
          ', '.join(text['mobile_number']), text['email'][0],
          text['website'][0], text['address'][0], text['city'][0],
          text['state'], text['pin_code'][0], text['image']
      )
      c.execute("INSERT INTO details VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", values)
      conn.commit()


      c.execute("SELECT * FROM details")
      rows = c.fetchall()

      sql_data = []
      for row in rows:
          sql_data.append(row)
      columns = ['card_holder','company_name', 'designation','mobile_number','email',
        'website', 'address','city', 'state','pin_code','image']


      df = pd.DataFrame(sql_data, columns=columns)



      return df

### Streamlit

conn = sqlite3.connect('biz_card.db')
c = conn.cursor()

st.set_page_config(page_title="Business Card Extraction & Visualization",layout='wide')


#=========hide the streamlit main and footer
hide_default_format = """
       <style>
       #MainMenu {visibility: hidden; }
       footer {visibility: hidden;}
       </style>
       """
st.markdown(hide_default_format, unsafe_allow_html=True)

#application background
def app_bg():
    st.markdown(f""" <style>.stApp {{
                        background: url("https://wallpapersafari.com/image/the-paper-wall-wallpapers.jpg");
                        background-size: cover}}
                     </style>""",unsafe_allow_html=True)
app_bg()


#### option menu
navigation,text_process=st.columns([1.2,4.55])
with navigation:
  selected = option_menu('Main Menu', ['Home',"Image to Text","Database"],
                        icons=["house",'file-earmark-font','gear'],default_index=0)






with text_process:
  if selected == 'Home':

    st.write('### TECHNOLOGIES USED')
    st.write('##### *:red[Python]  *:red[Streamlit] *:red[EasyOCR]  *:red[SQLite]')

    st.markdown("### Welcome to the Business Card Application!")
    st.markdown('###### Bizcard is a Python application designed to extract information from business cards. It utilizes various technologies such as :blue[Streamlit, Python, EasyOCR , RegEx function,and SQLite] database to achieve this functionality.')
    st.write('The main purpose of Bizcard is to automate the process of extracting key details from business card images, such as the name, designation, company, contact information, and other relevant data. By leveraging the power of OCR (Optical Character Recognition) provided by EasyOCR, Bizcard is able to extract text from the images.')
    st.write("Click on the ****:red[Image to text]**** option to start exploring the Bizcard extraction.")






  if selected == 'Image to Text':

    st.header(":violet[Business Card Extraction & Visualization]")

    st.subheader(":green[1.Extract & Insert Business Card]")

    uploaded_image = st.file_uploader("Choose an image file", type=["jpg", "jpeg", "png"])
    if st.button("Extract & Insert"):
        card_details = upload_get_data(uploaded_image)
        st.dataframe(card_details)
        image_size = (800, 800)

        image = Image.open(io.BytesIO(card_details['image']))
        image.thumbnail(image_size)
        st.image(image)


        all_df = get_data_to_sql(card_details)
        st.write("Card Extracted & Inserted Successfully")
  #### view existing data in database
  if selected == 'Database':
    try:
      st.subheader("Data Exists in Database")
      conn = sqlite3.connect('biz_card.db')
      c = conn.cursor()

      c.execute("SELECT * FROM details")
      rows = c.fetchall()
      all = []
      for row in rows:
        all.append(row)
      columns = ['Name','Company_Name','Designation','Mobile_number','Email','Website', 'Address','City', 'State','Pin_code','Image']
      all_df = pd.DataFrame(all,columns=columns)
      all_df['ID'] = range(1, len(all_df) + 1)
      st.dataframe(all_df[['ID','Company_Name', 'Name', 'Designation','Mobile_number','Email','Website', 'Address','City', 'State','Pin_code','Image']], hide_index=True)

      #### select a data for modify details in the database
      st.subheader(":green[Modify Inserted Business Card]")
      col1,col2 = st.columns([2,1])

      conn = sqlite3.connect('biz_card.db')
      c = conn.cursor()
      c.execute("SELECT card_holder FROM details")
      rows = c.fetchall()
      names = []
      for row in rows:
        names.append(row)
      columns = ['Name']
      df = pd.DataFrame(names,columns=columns)




      with col1:
        selected_name = st.selectbox(':violet[Select a Card_holder for Modify Card_info :]', df['Name'].unique())
        df_3 = all_df[all_df["Name"] == selected_name]
        df_4 = df_3.copy()

        st.subheader("Modify Data")

        new_name = st.text_input("Name", df_4["Name"].unique()[0])
        new_company = st.text_input("Company_Name", df_4["Company_Name"].unique()[0])
        new_design = st.text_input("Designation", df_4["Designation"].unique()[0])
        new_mobile = st.text_input("Mobile_number", df_4["Mobile_number"].unique()[0])
        new_email = st.text_input("Email", df_4["Email"].unique()[0])
        new_website = st.text_input("Website", df_4["Website"].unique()[0])
        new_addre = st.text_input("Address", df_4["Address"].unique()[0])
        new_city = st.text_input("City", df_4["City"].unique()[0])
        new_state = st.text_input("State", df_4["State"].unique()[0])
        new_pincode = st.text_input("Pincode", df_4["Pin_code"].unique()[0])

        if st.checkbox("View Image"):
          image_size = (800, 800)
          image = Image.open(io.BytesIO(df_4["Image"].unique()[0]))
          image.thumbnail(image_size)
          new_image = st.image(image)

        if st.button("Modify"):
          conn = sqlite3.connect('biz_card.db')
          c = conn.cursor()
          c.execute(f"UPDATE details SET card_holder = '{new_name}' WHERE card_holder = '{selected_name}'")
          c.execute(f"UPDATE details SET company_name = '{new_company}' WHERE card_holder = '{selected_name}'")
          c.execute(f"UPDATE details SET designation = '{new_design}' WHERE card_holder = '{selected_name}'")
          c.execute(f"UPDATE details SET mobile_number = '{new_mobile}' WHERE card_holder = '{selected_name}'")
          c.execute(f"UPDATE details SET email = '{new_email}' WHERE card_holder = '{selected_name}'")
          c.execute(f"UPDATE details SET website = '{new_website}' WHERE card_holder = '{selected_name}'")
          c.execute(f"UPDATE details SET address = '{new_addre}' WHERE card_holder = '{selected_name}'")
          c.execute(f"UPDATE details SET city = '{new_city}' WHERE card_holder = '{selected_name}'")
          c.execute(f"UPDATE details SET state = '{new_state}' WHERE card_holder = '{selected_name}'")
          c.execute(f"UPDATE details SET pin_code = '{new_pincode}' WHERE card_holder = '{selected_name}'")
          conn.commit()

          st.success("Modified Successfully")
          st.write(":red[Note: Refresh Page for view the modified data in the database]")




        st.subheader(":green[Delete Business Card]")

        conn = sqlite3.connect('biz_card.db')
        c = conn.cursor()
        c.execute("SELECT card_holder FROM details")
        rows = c.fetchall()
        columns = ['Name']
        dele_df = pd.DataFrame(rows,columns=columns)
        selected_name = st.selectbox('Select a Name for delete :', dele_df['Name'].unique())

        if st.button("Delete"):
          c.execute(f"DELETE FROM details WHERE card_holder = '{selected_name}'LIMIT 1")
          conn.commit()
          st.success("Selected Record Deleted Successfully")
    except:
      st.warning("Please Upload the Business Card for Extraction")





Overwriting app.py


In [13]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.168.193.157
npx: installed 22 in 2.472s
your url is: https://clever-cougars-grab.loca.lt
